In [1]:
from src import Tensor,cross_entropy,Adam,SGD,Linear, Module, Sequential, ReLU,dataloader,Conv2D
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [4]:
class Net(Module):
    def __init__(self, inc, outc, impl):
        super().__init__()
        self.seq = Sequential(
            Conv2D(inc, 64, 3, conv_impl=impl),
            ReLU(),
            lambda x: x.reshape(x.shape[0], -1),
            Linear(2304, outc,bias=False)
        )

    def forward(self, x):
        x = self.seq(x)
        x = x.reshape(x.shape[0],-1)
        return x


def train(net,optim,train_X,train_y,epochs):
    if train_X.ndim == 2:
        h=w=int(np.sqrt(train_X.shape[1]))
        train_X = train_X.reshape(train_X.shape[0],1,h,w)
    import time
    start = time.monotonic()
    for epoch in range(epochs):  # loop over the dataset multiple times
        losses = []
        accs = []
        for X, y in dataloader(64*2, train_X, train_y):
            # forward + backward + optimize
            outputs = net.forward(X)
            loss = cross_entropy(outputs, y)
            optim.zero_grad()
            loss.backward()
            assert loss.item() >= 0, loss
            optim.step()

            losses.append(loss.item())
            accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
        print(
            f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):.3f} accuracy: {np.mean(accs)*100:3.2f}')
    return time.monotonic() - start

In [5]:
net = Net(1, np.unique(train_y).size, impl="slow")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 61.879 accuracy: 48.41
[Epoch   2] loss: 10.776 accuracy: 83.67
[Epoch   3] loss: 4.572 accuracy: 92.76
[Epoch   4] loss: 1.929 accuracy: 96.46
[Epoch   5] loss: 1.087 accuracy: 97.55
[Epoch   6] loss: 0.729 accuracy: 98.12
[Epoch   7] loss: 0.413 accuracy: 98.39
[Epoch   8] loss: 0.266 accuracy: 98.75
[Epoch   9] loss: 0.187 accuracy: 99.17
[Epoch  10] loss: 0.159 accuracy: 99.32
Finished Training in : 16.2030 s


In [6]:
net = Net(1, np.unique(train_y).size, impl="fast_forward")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 55.173 accuracy: 49.92
[Epoch   2] loss: 6.965 accuracy: 89.74
[Epoch   3] loss: 5.139 accuracy: 93.20
[Epoch   4] loss: 2.280 accuracy: 95.57
[Epoch   5] loss: 1.394 accuracy: 96.98
[Epoch   6] loss: 0.857 accuracy: 98.18
[Epoch   7] loss: 0.505 accuracy: 98.49
[Epoch   8] loss: 0.304 accuracy: 98.96
[Epoch   9] loss: 0.260 accuracy: 98.91
[Epoch  10] loss: 0.168 accuracy: 99.48
Finished Training in : 11.2810 s


In [7]:
net = Net(1, np.unique(train_y).size, impl="fast")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 54.194 accuracy: 45.42
[Epoch   2] loss: 10.706 accuracy: 86.77
[Epoch   3] loss: 3.813 accuracy: 93.91
[Epoch   4] loss: 2.438 accuracy: 95.02
[Epoch   5] loss: 3.253 accuracy: 94.27
[Epoch   6] loss: 1.434 accuracy: 97.08
[Epoch   7] loss: 0.912 accuracy: 96.17
[Epoch   8] loss: 1.450 accuracy: 95.99
[Epoch   9] loss: 0.884 accuracy: 97.45
[Epoch  10] loss: 0.383 accuracy: 98.49
Finished Training in : 2.6090 s
